In [2]:
import os
import matplotlib.pyplot as plt
import numpy as np
from sklearn.model_selection import learning_curve
from tqdm import tqdm

from sklearn.datasets import load_files
from pyvi import ViTokenizer

from sklearn import svm
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score, confusion_matrix, ConfusionMatrixDisplay

%matplotlib inline

INPUT = 'data/news_vnexpress'
os.makedirs("images",exist_ok=True)  # thư mục lưu các hình ảnh kết quả trong quá trình huấn luyện và đánh giá

# statistics
print('Các nhãn và số văn bản tương ứng trong dữ liệu')
print('----------------------------------------------')
n = 0
for label in os.listdir(INPUT):
    print(f'{label}: {len(os.listdir(os.path.join(INPUT, label)))}')  #duyet qua thu muc INPUT va in ten cac thu muc con va sl tep
    n += len(os.listdir(os.path.join(INPUT, label)))

print('-------------------------')
print(f"Tổng số văn bản: {n}")

# load data
data_train = load_files(container_path=INPUT, encoding="utf-8")

print('mapping:')
for i in range(len(data_train.target_names)):
    print(f'{data_train.target_names[i]} - {i}')

print('-----------Dữ liệu của bài viết đầu tiên-----------')
print('tên file:', data_train.filenames[0:1])
print('data_train:', data_train.data[0:1])
print('target:', data_train.target[0:1])

print("\nTổng số  văn bản: {}" .format( len(data_train.filenames)))

# load dữ liệu các stopwords 
with open("data/vietnamese-stopwords.txt", encoding="utf-8") as f:
    stopwords = f.readlines()
stopwords = [x.strip().replace(" ", "_") for x in stopwords]    #strip() sẽ loại bỏ khoảng trắng 2 phía và replace để thay thế khoảng trắng bằng dấu _
print(f"Số lượng stopwords: {len(stopwords)}")
print(stopwords[:10])

# Chuyển hoá dữ liệu text về dạng vector TF print(module_count_vector)
#     - loại bỏ từ dừng
#     - sinh từ điển
module_count_vector = CountVectorizer(stop_words=stopwords) #chỉnh định từ cần loại bỏ là cac stopword từ file vua doc
model_rf_preprocess = Pipeline([('vect', module_count_vector),('tf', TfidfTransformer()),]) #truyền vào các bước thực hiện tuần tự

# Hàm thực hiện chuyển đổi dữ liệu text thành dữ liệu số dạng ma trận 
# Input: Dữ liệu 2 chiều dạng numpy.array, mảng nhãn id dạng numpy.array 
data_preprocessed = model_rf_preprocess.fit_transform(data_train.data, data_train.target)
# In ra một mẫu dữ liệu văn bản trước khi xử lý
print("Dữ liệu văn bản trước khi xử lý:")
print(data_train.data[0])

# In ra một mẫu dữ liệu văn bản sau khi xử lý
print("\nDữ liệu văn bản sau khi xử lý:")
print(data_preprocessed[0])

#0 là chỉ số của mẫu văn bản.
#12711 là chỉ số của từ trong từ điển. Mỗi từ trong từ điển sẽ được gán một chỉ số.
#0.47047455226652657 là giá trị TF-IDF của từ tương ứng (có chỉ số là 12711) trong mẫu văn bản đó.
#nó có nhiều từ mà chỉ còn vài dòng là do mốt số từ đã bị loại do có chỉ số TFIDF sap si bang 0

print(f"\nSố lượng từ trong từ điển: {len(module_count_vector.vocabulary_)}")
print(f"Kích thước dữ liệu sau khi xử lý: {data_preprocessed.shape}")
print(f"Kích thước nhãn tương ứng: {data_train.target.shape}")

#chia 
pivot = int(data_preprocessed.shape[0] * 0.8) # shape[0] la dong, 1 la cot, ma tran thua thì nó bo may cai cot co gia tri la 0 roi
print('pivot:',pivot)
X_train, X_test = data_preprocessed[0:pivot], data_preprocessed[pivot:]
print('X')
Y_train, Y_test = data_train.target[0:pivot], data_train.target[pivot:]

#Lựa chọn (tối ưu) tham số
def cross_validation(estimator): #tham số estimator là mô hình sẽ đưa vào
    _, train_scores, test_scores = learning_curve(estimator, X_train, Y_train, cv=10, n_jobs=-1, train_sizes=[1.0, ], scoring="accuracy")
    test_scores = test_scores[0]
    mean, std = test_scores.mean(), test_scores.std()
    return mean, std

def plot(title, xlabel, X, Y, error, ylabel = "Accuracy"):
    plt.xlabel(xlabel)
    plt.title(title)
    plt.grid()
    plt.ylabel(ylabel)
    plt.errorbar(X, Y, error, linestyle='None', marker='o')


#test
svm_ = svm.SVC(kernel='linear', C=1.0)
rf = RandomForestClassifier(criterion='gini', n_estimators=100)

# Huấn luyện các mô hình trên tập dữ liệu train đầy đủ
svm_.fit(X_train, Y_train)
rf.fit(X_train, Y_train)

# Kết quả dự đoán trên tập test
print(f'SVM: {accuracy_score(Y_test, svm_.predict(X_test))}')
print(f'RF: {accuracy_score(Y_test, rf.predict(X_test))}')


Các nhãn và số văn bản tương ứng trong dữ liệu
----------------------------------------------
doi-song: 120
du-lich: 54
giai-tri: 201
giao-duc: 105
khoa-hoc: 144
kinh-doanh: 262
phap-luat: 59
suc-khoe: 162
the-thao: 173
thoi-su: 59
-------------------------
Tổng số văn bản: 1339
mapping:
doi-song - 0
du-lich - 1
giai-tri - 2
giao-duc - 3
khoa-hoc - 4
kinh-doanh - 5
phap-luat - 6
suc-khoe - 7
the-thao - 8
thoi-su - 9
-----------Dữ liệu của bài viết đầu tiên-----------
tên file: ['data/news_vnexpress\\khoa-hoc\\00133.txt']
data_train: ['Mời độc giả đặt câu hỏi tại đây\n']
target: [4]

Tổng số  văn bản: 1339
Số lượng stopwords: 2063
['a_lô', 'a_ha', 'ai', 'ai_ai', 'ai_nấy', 'ai_đó', 'alô', 'amen', 'anh', 'anh_ấy']
Dữ liệu văn bản trước khi xử lý:
Mời độc giả đặt câu hỏi tại đây


Dữ liệu văn bản sau khi xử lý:
  (0, 12711)	0.47047455226652657
  (0, 7462)	0.6112865759350837
  (0, 4378)	0.44175809334129823
  (0, 3084)	0.4580744532481954

Số lượng từ trong từ điển: 12796
Kích thước dữ liệu s